In [16]:
import os
import requests
from bs4 import BeautifulSoup
import openai
from telegram import Update, Bot
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters
from dotenv import load_dotenv
import nest_asyncio

In [17]:
# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID")

openai.api_key = OPENAI_API_KEY


In [18]:
def fetch_bbc_news():
    url = "https://www.bbc.com/news"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    articles = []
    for h2 in soup.find_all("h2", attrs={"data-testid": "card-headline"}):
        title = h2.get_text(strip=True)

        # Try to find the <a> tag associated with the headline
        a_tag = h2.find_parent("a")
        if not a_tag:
            a_tag = h2.find_next("a")

        if a_tag and a_tag.has_attr("href"):
            href = a_tag["href"]
            link = "https://www.bbc.com" + href if href.startswith("/") else href

            # Fetch and extract article text
            try:
                article_response = requests.get(link, headers=headers)
                article_soup = BeautifulSoup(article_response.content, "html.parser")
                paragraphs = article_soup.find_all("p")
                content = "\n".join(p.get_text(strip=True) for p in paragraphs)
                if content:
                    articles.append({
                        "title": title,
                        "link": link,
                        "content": content
                    })
            except Exception as e:
                print(f"Error fetching article: {e}")

        if len(articles) >= 3:
            break

    return articles


In [19]:
system_prompt = (
    "You are an assistant that receives the contents of a news website and its title; "
    "your task is to summarize the contents of the website."
)

def define_user_prompt(article): 
    user_prompt = f"You are given the news website titled: {article['title']}\n"
    user_prompt += (
        "The contents of the website are as follows; please provide "
        "a summary of the news website in less than 100 words. \n\n"
    )
    user_prompt += article['content']
    return user_prompt 

def openai_message(article): 
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': define_user_prompt(article)}
    ]


In [20]:
# TASK -- Pass the message to an OpenAI model and extract the completion 
def summarize(article): 
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini', 
        messages = openai_message(article)
        )
    return response.choices[0].message.content

# Example usage
#url_summary = summarize(article)


In [42]:
print(url_summary)

The news article highlights the comments made by U.S. diplomat Marco Rubio regarding the upcoming peace talks between Ukraine and Russia scheduled to take place in Istanbul, Turkey. Rubio expresses skepticism about the potential for significant progress in these negotiations without direct interaction between former U.S. President Donald Trump and Russian President Vladimir Putin. Both Rubio and Trump noted that effective dialogue is contingent upon a meeting between the two leaders. 

Ukrainian President Volodymyr Zelensky confirmed that Ukraine would send a delegation to the talks but criticized Russia's "low-level" delegation, headed by presidential aide Vladimir Medinsky, as disrespectful to both President Trump and Turkish President Recep Tayyip Erdoğan. Zelensky challenged Putin to a personal meeting, emphasizing the lack of high-level representation from the Kremlin as a sign of disrespect.

The Turkish, U.S., Ukrainian, and Russian delegations had been set to meet in Istanbul f

In [ ]:
nest_asyncio.apply()
# Telegram handler to respond to "news" messages
async def news_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    msg_text = update.message.text.lower()
    if "news" in msg_text:
        await update.message.reply_text("Fetching the latest BBC news and summarizing...")
        articles = fetch_bbc_news()
        summaries = []
        for article in articles:
            try:
                summary = summarize(article)  # Assuming summarize is async and takes article as param
                summaries.append(f"*{article['title']}*\n{article['link']}\n{summary}\n\n")
            except Exception as e:
                summaries.append(f"Could not summarize article '{article['title']}': {e}\n\n")
        reply_text = "\n".join(summaries) or "Sorry, no news articles found."
        await update.message.reply_text(reply_text, parse_mode='Markdown')
    else:
        await update.message.reply_text("Hi! Send me a message containing the word 'news' to get BBC news summaries.")


# Optional: start command
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Hi! Send me a message containing the word 'news' and I'll fetch summaries from BBC News for you.")

def main():
    app = ApplicationBuilder().token(TELEGRAM_BOT_TOKEN).build()

    # Command handler for /start
    app.add_handler(CommandHandler("start", start))

    # Message handler for messages containing "news" (case insensitive)
    app.add_handler(MessageHandler(filters.TEXT & (~filters.COMMAND), news_handler))

    print("Bot started...")
    app.run_polling(close_loop=False)

if __name__ == "__main__":
    main()

Bot started...
